## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit and the 1900-unit architecture. 

We recommend getting started with the 64-unit architecture as it is easier and faster to run, but has the same interface as the 1900-unit one.

Use the 64-unit or the 1900-unit model?

In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

## Setup

In [2]:
import tensorflow as tf
import numpy as np

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

## Data formatting and management

Initialize UniRep, also referred to as the "babbler" in our code. You need to provide the batch size you will use and the path to the weight directory.

In [3]:
batch_size = 12
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


UniRep needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

In [11]:
seq = "MRKGEELFTGVVPILVELDGDVNGHKFSVRGFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

In [12]:
np.array(b.format_seq(seq))

array([24,  1,  2,  4, 13,  6,  6, 21, 18,  8, 13, 16, 16, 14, 17, 21, 16,
        6, 21,  5, 13,  5, 16,  9, 13,  3,  4, 18,  7, 16,  2, 13, 18,  4,
        7, 15,  1, 14,  6, 13, 19, 16, 10,  6,  2,  8, 17,  7, 18,  4,  5,
        5, 13,  8, 19,  4,  8,  2, 15,  6, 16,  4, 18,  6, 13,  5,  8, 21,
       16,  9,  2, 17,  6, 21,  4, 13, 17,  5, 18,  4,  6,  5, 13,  9, 17,
       21, 13,  3,  4, 21,  6, 19,  9, 18,  9,  7,  3,  9, 16, 19, 17,  8,
       15,  5,  4, 10,  4,  9, 13, 17,  4, 15,  9, 18,  4, 17,  2,  3,  9,
       16,  6,  5, 13,  7, 16, 10, 21, 15,  5,  3, 19, 10, 10,  9,  8, 14,
       17, 13,  5, 13, 14, 16, 21, 21, 14,  5,  9,  3, 19, 21,  7,  8, 10,
        7, 16, 21,  7,  4,  5, 14,  9,  6,  4,  2,  5,  3,  1, 16, 21, 21,
        6, 18, 16,  8, 15, 15, 13, 17,  8,  3, 13,  1,  5,  6, 21, 19,  4])

We also provide a function that will check your amino acid sequences don't contain any characters which will break the UniRep model.

In [13]:
b.is_valid_seq(seq)

True

In [14]:
rep = b.get_rep(seq)
print(rep[-1])
print(len(rep[-1]))

[-7.1467146e-02  8.5657609e-01 -3.3489746e-01 -9.0909624e+00
 -5.1293910e-02 -7.9396874e-01  1.2415814e+00  1.3914506e+00
  4.2770794e-01 -9.1309536e-01  7.3714471e-01  3.7204704e+00
 -6.1502516e-02  3.6691222e-02 -5.3945884e-02 -5.6928349e-01
 -8.8524551e+00  6.2627241e-02  1.7972789e+00 -4.4495778e+00
 -3.1581536e-01  2.1176586e+00 -1.6655995e+00 -3.0214908e+00
  5.4951391e+00 -1.0015080e+00 -2.7671546e-01  1.5090264e-01
  6.7902198e+00 -1.1953800e+01  5.2513447e+00  5.0253266e-01
  1.6759138e+00 -7.7483821e-01  1.5462296e+00  3.3909041e-01
 -6.6190827e-01  3.6993891e-01  2.9829463e-02 -5.7673329e-01
  2.1338558e-02  5.3747725e-01 -4.6976127e-02  7.2446221e-01
  1.1045490e-01  2.6462986e+00 -2.1396317e+00 -1.6925581e+01
  1.5338727e+00  8.3889019e-01  7.0084685e-01  1.8553182e-01
 -2.1598063e-01 -2.4932203e-01  1.6302848e-01  6.0178697e-02
 -1.1138448e-01 -2.4317503e-02  2.5652668e-02  1.0783783e+01
 -4.2227337e-01 -3.4802705e-02  3.2718506e+00  4.0632207e-03]
64


In [0]:
b.get_babble("MRKGEEL", length=290, temp=0.3)

In [18]:
import pandas as pd
import numpy as np
from unirep import babbler64 as babbler


def get_data(filename):
    df_pet = pd.read_csv(filename)
    seqs = df_pet["aaSequence"]
    vals = df_pet["medianBrightness"]
    vals = vals.astype(np.float)
    return seqs, vals


# Turns each sequence into a np array
def vectorize_data():
    seqs, vals = get_data("data/PETase_mutations_1.csv")
    seq_vecs = []
    for seq in seqs:
        seq = seq.upper()
        seq_vecs.append(b.get_rep(seq))
    seq_vecs = np.array(seq_vecs)
    np.save("data/petase_seqvecs.npy", seq_vecs)
    np.save("data/petase_vals.npy", vals)

    seqs, vals = get_data("data/hydrolase.csv")
    seq_vecs = []
    for i, seq in enumerate(seqs):
        if i % 25 == 0:
            print(i, "seqs processed...")
        seq = seq.upper()
        seq_vecs.append(b.get_rep(seq))
    seq_vecs = np.array(seq_vecs)
    np.save("data/hydrolase_seqvecs.npy", seq_vecs)
    np.save("data/hydrolase_vals.npy", vals)

vectorize_data()

0 seqs processed...
25 seqs processed...


KeyboardInterrupt: 

# Instability

In [0]:
AMINO_ACIDS = ['W', 'C', 'M', 'H', 'Y', 'F', 'Q', 'N', 'I', 'R', 'D', 'P', 'T',
               'K', 'E', 'V', 'S', 'G', 'A', 'L']

def construct_diwv():
  diwv_dict = {}
  matrix = []
  with open("./diwv.csv", "r") as f:
    for line in f.readlines():
      scores = line.split(",")
      matrix.append([float(s) for s in scores])
  for i in range(20):
    subdict = {}
    for j in range(20):
      subdict[AMINO_ACIDS[j]] = matrix[i][j]
    diwv_dict[AMINO_ACIDS[i]] = subdict
  return diwv_dict
  
diwv_dict = construct_diwv()

# Karpathy RNN

In [0]:
from oracles import oracle

# Returns True iff protein sequence passes preset filters
def passes_filters(seq, min_len=270, max_len=310,
                   max_instability_index=40,
                   max_conserved_residue_penalty=20,
                   oracle_name='deepnn'):

  # controls sequence length
  if len(seq) <= 270 or len(seq) >= 310:
    return False
  
  # controls sequence stability (ProtParam)
  score = 0
  for i in range(len(seq)-1):
    score += diwv_dict[seq[i]][seq[i+1]]
    score = 10.0/len(seq) * score
  if score > 40.0:
    return False

  # ensures optimal sequence function using oracles
  o = oracle
  if o.predict(oracle_name, seq):
    return True
  return False

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

def run_RNN(seq_length=100, BATCH_SIZE=64, BUFFER_SIZE=10000,
            embedding_dim=256, rnn_units=1024, num_generate=300,
            temperature=0.1, seed=u"MNFPRASRLMQAAVL",
            path_to_file="./seqdump_processed.txt"):
  # Read, then decode for py2 compat.
  text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
  # length of text is the number of characters in it
  print ('Length of text: {} characters'.format(len(text)))

  vocab = sorted(set(text))
  print('{} unique characters'.format(len(vocab)))

  char2idx = {c:i for i,c in enumerate(vocab)}
  idx2char = np.array(vocab)

  text_as_int = np.array([char2idx[c] for c in text])

  print('{} ---map---> {}'.format(text[:13], text_as_int[:13]))

  
  examples_per_epoch = len(text) // seq_length

  char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

  sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

  def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

  dataset = sequences.map(split_input_target)

  for input_example, target_example in dataset.take(1):
    print('Input: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target: ', repr(''.join(idx2char[target_example.numpy()])))


  dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
  dataset

  vocab_size = len(vocab)

  def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, 
                             stateful=True, 
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

  model = build_model(
      vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
  )

  model.summary()

#   for input_example_batch, target_example_batch in dataset.take(1):
#     example_batch_predictions = model(input_example_batch)
#     print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

#   sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
#   sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

#   print("Input: \n", repr(''.join(idx2char[input_example_batch[0]])))
#   print("Output: \n", repr(''.join(idx2char[sampled_indices])))   # powerful np.array indexing ability

  def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#   example_batch_loss = loss(target_example_batch, example_batch_predictions)
#   print("Prediction shape: ", example_batch_loss.shape, "# (batch_size, sequence_length)")
#   print("scalar_loss:      ", example_batch_loss.numpy().mean())

  model.compile(optimizer='adam', loss=loss)

  checkpoint_dir = './training_checkpoints'
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
  checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_prefix,
      save_weights_only=True)
  EPOCHS=30


  history = model.fit(dataset, epochs=EPOCHS, 
                      callbacks=[checkpoint_callback])

  model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
  model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
  model.build(tf.TensorShape([1, None]))
  model.summary()

  def generate_text():
    
    input_eval = [char2idx[c] for c in seed]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []


    model.reset_states()
    for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

    return (seed + ''.join(text_generated))
  
  valid_seqs = []
  while len(valid_seqs) < 5:
    blabble = generate_text()
    seq = blabble.split("\n")[0]
    if not passes_filters(seq.strip()):
      continue
    else:
      valid_seqs.append(seq)
      
  return valid_seqs
  

In [0]:
def refresh_training_set(iter_num):
  valid_seqs = run_RNN(iter_num)
  
  with open("seqdump_processed_" + str(iter_num) + ".txt", "r") as f:
    with open("seqdump_processed_" + str(iter_num+1) + ".txt", "w") as g:
      prev_seqs = f.readlines().strip()
      new_seqs = prev_seqs + "\n\n"
      for seq in valid_seqs:
        new_seqs += seq + "\n\n"
      g.write(new_seqs)

for i in range(10):
  refresh_training_set(i)

# Training

You could use your own data flow as long as you ensure that the data format is obeyed. Alternatively, you can use the data flow we've implemented for UniRep training, which happens in the tensorflow graph. It reads from a file of integer sequences, shuffles them around, collects them into groups of similar length (to minimize padding waste) and pads them to the max_length. Here's how to do that:

First, sequences need to be saved in the correct format. Suppose we have a new-line seperated file of amino acid sequences, `seqs.txt`, and we want to format them. Note that training is currently only publicly supported for amino acid sequences less than 275 amino acids as gradient updates for sequences longer than that start to get unwieldy. If you want to train on sequences longer than this, please reach out to us. 

Sequence formatting can be done as follows:

In [0]:
# Before you can train your model, 
with open("seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

This is what the integer format looks like

In [0]:
!head -n1 formatted.txt

24,1,2,4,13,6,6,21,18,8,13,16,16,14,17,21,16,6,21,5,13,5,16,9,13,3,4,18,7,16,2,13,6,13,6,13,5,15,8,9,13,4,21,8,21,4,18,17,11,8,8,13,4,21,14,16,14,20,14,8,21,16,8,8,21,8,19,13,16,10,11,18,15,2,19,14,5,3,1,4,10,3,5,18,18,4,7,15,1,14,6,13,19,16,10,6,2,8,17,7,18,4,5,5,13,8,19,4,8,2,15,6,16,4,18,6,13,5,8,21,16,9,2,17,6,21,4,13,17,5,18,4,6,5,13,9,17,21,13,3,4,21,6,19,9,18,9,7,3,9,16,19,17,8,15,5,4,10,4,9,13,17,4,15,9,18,4,17,2,3,9,16,6,5,13,7,16,10,21,15,5,3,19,10,10,9,8,14,17,13,5,13,14,16,21,21,14,5,9,3,19,21,7,8,10,7,16,21,7,4,5,14,9,6,4,2,5,3,1,16,21,21,6,18,16,8,15,15,13,17,8,3,13,1,5,6,21,19,4


Notice that by default format_seq does not include the stop symbol (25) at the end of the sequence. This is the correct behavior if you are trying to train a top model, but not if you are training UniRep representations.

Now we can use a custom function to bucket, batch and pad sequences from `formatted.txt` (which has the correct integer codex after calling `babbler.format_seq()`). The bucketing occurs in the graph. 

What is bucketing? Specify a lower and upper bound, and interval. All sequences less than lower or greater than upper will be batched together. The interval defines the "sides" of buckets between these bounds. Don't pick a small interval for a small dataset because the function will just repeat a sequence if there are not enough to
fill a batch. All batches are the size you passed when initializing the babbler.

This is also doing a few other things:
- Shuffling the sequences by randomly sampling from a 10000 sequence buffer
- Automatically padding the sequences with zeros so the returned batch is a perfect rectangle
- Automatically repeating the dataset

In [0]:
bucket_op = b.bucket_batch_pad("formatted.txt", interval=1000) # Large interval

Inconveniently, this does not make it easy for a value to be associated with each sequence and not lost during shuffling. You can get around this by just prepending every integer sequence with the sequence label (eg, every sequence would be saved to the file as "{brightness value}, 24, 1, 5,..." and then you could just index out the first column after calling the `bucket_op`. Please reach out if you have questions on how to do this.

Now that we have the `bucket_op`, we can simply `sess.run()` it to get a correctly formatted batch

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)
    
print(batch)
print(batch.shape)

[[24  1  6 ... 13 18  6]
 [24  1  7 ...  0  0  0]
 [24  1  7 ...  0  0  0]
 ...
 [24  1  4 ...  0  0  0]
 [24  1 10 ...  0  0  0]
 [24  1 13 ...  0  0  0]]
(12, 265)


You can look back and see that the batch_size we passed to __init__ is indeed 12, and the second dimension must be the longest sequence included in this batch. Now we have the data flow setup (note that as long as your batch looks like this, you don't need my flow), so we can proceed to implementing the graph. The module returns all the operations needed to feed in sequence and get out trainable representations.

## Training a top model and a top model + mLSTM.

First, obtain all of the ops needed to output a representation

In [0]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

`final_hidden` should be a batch_size x rep_dim matrix.

Lets say we want to train a basic feed-forward network as the top model, doing regression with MSE loss, and the Adam optimizer. We can do that by:

1.  Defining a loss function.

2.  Defining an optimizer that's only optimizing variables in the top model.

3.  Minimizing the loss inside of a TensorFlow session

In [0]:
y_placeholder = tf.placeholder(tf.float32, shape=[None,1], name="y")
initializer = tf.contrib.layers.xavier_initializer(uniform=False)

with tf.variable_scope("top"):
    prediction = tf.contrib.layers.fully_connected(
        final_hidden, 1, activation_fn=None, 
        weights_initializer=initializer,
        biases_initializer=tf.zeros_initializer()
    )

loss = tf.losses.mean_squared_error(y_placeholder, prediction)

You can specifically train the top model first by isolating variables of the "top" scope, and forcing the optimizer to only optimize these.

In [0]:
learning_rate=.001
top_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="top")
optimizer = tf.train.AdamOptimizer(learning_rate)
top_only_step_op = optimizer.minimize(loss, var_list=top_variables)
all_step_op = optimizer.minimize(loss)

We next need to define a function that allows us to calculate the length each sequence in the batch so that we know what index to use to obtain the right "final" hidden state

In [0]:
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)
    return lengths

nonpad_len(batch)

array([265, 195, 239, 176, 230, 251,  97, 223, 148, 223, 248, 205])

We are ready to train. As an illustration, let's learn to predict the number 42 just optimizing the top model.

In [0]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, top_only_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
                  
        print("Iteration {0}: {1}".format(i, loss_))

Iteration 0: 1757.400390625
Iteration 1: 1763.5225830078125
Iteration 2: 1736.2425537109375
Iteration 3: 1744.5452880859375
Iteration 4: 1726.3717041015625
Iteration 5: 1711.666015625
Iteration 6: 1719.5970458984375
Iteration 7: 1693.2880859375
Iteration 8: 1701.9228515625
Iteration 9: 1683.3359375


We can also jointly train the top model and the mLSTM. Note that if using the 1900-unit (full) model, you will need a GPU with at least 16GB RAM. To see a demonstration of joint training with fewer computational resources, please run this notebook using the 64-unit model.

In [0]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, all_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
        
        print("Iteration {0}: {1}".format(i,loss_))

Iteration 0: 1757.400390625
Iteration 1: 1720.1832275390625
Iteration 2: 1650.4283447265625
Iteration 3: 1552.2196044921875
Iteration 4: 1369.4599609375
Iteration 5: 1174.6700439453125
Iteration 6: 981.3558959960938
Iteration 7: 798.6965942382812
Iteration 8: 715.9775390625
Iteration 9: 580.2219848632812
